### Installing dependencies

In [ ]:
pip install beautifulsoup4

In [ ]:
import selenium
# import ipython
import time
import parsel
import csv

import pandas as pd

In [ ]:
# import web driver
from selenium import webdriver




In [ ]:
#dfr = pd.read_csv('results_file.csv')

In [ ]:
import openpyxl


### We create a separate Excel sheet to write our fetched data

In [ ]:
res = openpyxl.load_workbook(filename='results_file.xlsx')
results = res.worksheets[0]


In [ ]:
#fr.head(10)

### Leads.csv contains information about the Individuals to be searched. Columns include "First Name", "Last Name" and "Company"

In [ ]:
df = pd.read_csv('leads.csv',index_col='ID')

In [ ]:
df.head(10)

### We combine first name and last name

In [ ]:
df['Name']=df['FirstName']+' '+df['LastName']

In [ ]:
df.head()

### Create two Lists to store Name of the individual and the Name of the Comapnies

In [ ]:
names=df["Name"].iloc[:].values
companies=df["Company"].iloc[:].values

#### Imported googlesearch api to automate query searching

### We google search the following query for every individual:
Eg. If an Individual's names is Jack Smith and he belongs to a company with name "Microsoft" then we will googlesearch <strong> "site:linkedin.com/in/ Jack Smith Microsoft" </strong> and fetch the URL of the <strong>first search result</strong>.
> The more input parameters we feed in this query apart from Name of the Individual and the organization he belongs to, higher the chances of uniquely identifying an individual. Since there can be many Jack Smith's working at Microsoft.

In [ ]:
from googlesearch import search 



In [ ]:
# to search 
col_name=1
col_company_name=8
col_person_link=11
links=[]
for i in range(len(companies)):
    query = 'site:linkedin.com/in/'+" " + names[i] +" "+ companies[i] 
    
    print("Query to search:",query)
    person_name=names[i]
    company_name=companies[i]
    for url in search(query, tld="co.in", num=1, stop=1,pause=5): 
        person_link=url
        print("LinkedIn URL:",person_link)
        results.cell(row=i+2, column=col_name).value = person_name
        results.cell(row=i+2, column=col_company_name).value = company_name
        results.cell(row=i+2, column=col_person_link).value = person_link
        links.append(person_link)
        



In [ ]:
res.save(filename='results_file.xlsx')

We create a separate .py file with variables:
- username = 'Your linkedin email id'
- Password = 'Your linkedin password'


In [ ]:
%run "parameters.py"

In [ ]:
import parameters

In [ ]:
# specifies the path to the chromedriver.exe
driver = webdriver.Chrome('Location of the file Webdriver.exe to be put here')

In [ ]:
# driver.get method() will navigate to a page given by the URL address
driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

In [ ]:
# locate email form by_class_name
username = driver.find_element_by_id('username')


# send_keys() to simulate key strokes
username.send_keys(parameters.linkedin_username)
# sleep for 0.5 seconds
time.sleep(0.5)

In [ ]:
# locate password form by_class_name
password = driver.find_element_by_id('password')

# send_keys() to simulate key strokes
password.send_keys(parameters.linkedin_password)
time.sleep(0.5)

In [ ]:

# locate submit button by_id
log_in_button = driver.find_element_by_class_name('login__form_action_container ')

# .click() to mimic button click
log_in_button.click()

In [ ]:
from parsel import Selector

In [ ]:
res.save(filename='results_file.xlsx')

In [ ]:
col_name=13
curr_title=2
prev_title=3





for i in range(len(links)): #Iterate loop for number of links
    j=0
    driver.get(links[i]) #Landed on LinkedIn profile of the person
    time.sleep(2.5) #Sleep to show that its not automated
    #text=driver.page_source 
    sel = Selector(text=driver.page_source) 
    person_location = sel.xpath('//*[starts-with(@class, "t-16 t-black t-normal inline-block")]/text()').extract_first()
    
    experience = driver.find_elements_by_css_selector('#experience-section .pv-profile-section')
    employer_title=[]
    for item in experience:
        j+=1
        if(j==3):
            break
        else:
            print("Experience: ",j)
            a=item.text.split('\n') 
            # Variable a is a list which stores all the information retrieved from the "Experience" section
            # of the LinkedIn page.
    
            if(a[0]=="Company Name"):
                employer_title.append((a[1],a[5]))
            else:
                employer_title.append((a[2],a[0]))
        print(employer_title)
    for k in range(len(employer_title)):
        results.cell(row=i+2, column=curr_title).value = employer_title[0][1]
        results.cell(row=i+2, column=prev_title).value = employer_title[1][1]
    
    

In [ ]:
res.save(filename='results_file.xlsx')